THIS CODE IS FOR MY PERSONAL PNL, RISK AND POSITION MONITORING, AND WORKS ONLY WITH LOCAL TRANSACTION HISTORY FILES
Other projects on my github are more scalable and in english, for this one it is in frenglish.

Code pour calculer mon PnL, mes positions, suivre mes risques.

On va prendre les data dans mon excel, sur l'onglet historique de transaction. 

On va créer une database SQL avec toutes nos data de YF price et on compute returns

on va juste prendre des closing day data.

In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import sqlite3
import numpy as np
import logging
import matplotlib.pyplot as plt

#Name of our local transaction inventory spreadsheet
excel_file = "nameofexcel.xlsx"
databasename = 'nameofsqldatabase.db'


In [ ]:
# Configure logging to suppress warnings and errors
logging.basicConfig(level=logging.INFO)  # Set to ERROR to suppress INFO messages
yf_logger = logging.getLogger('yfinance')
yf_logger.setLevel(logging.ERROR)  # Suppress warnings and errors from yfinance

In [ ]:
#Connexion to SQL database
conn = sqlite3.connect(databasename)
cursor = conn.cursor()

Ici on a ajouté une ligne de code pour reset la table SQL avec les données de ticker

A NE RUN QUE SI ON VEUT RESET NOTRE INDEX DE TICKER ET SHORTNAME

In [ ]:
def reset_database(database):
    cursor.execute("DELETE FROM ticker_info")
    conn.commit() 

reset_database(databasename)

In [ ]:
portfolio_tickers = pd.read_excel(excel_file, sheet_name="Data", usecols=[0, 1, 2, 3]).dropna(subset=['ISIN'])
#usecols=[0... 3 pour utiliser que les 4 premieres colonnes de l'excel
#drop toutes les lignes ou il y a une NaN value sur le ISIN (donc pas de ISIN)
transactions = pd.read_excel(excel_file, sheet_name="Transaction history")

We have created our SQL database and tables with the below structure, for daily data only:

cursor.execute('''
               CREATE TABLE IF NOT EXISTS ticker_info(
               product_index INTEGER PRIMARY KEY AUTOINCREMENT,
               YF_ticker TEXT,
               isin TEXT,
               longName TEXT,
               shortName TEXT,
               industry TEXT,
               sector TEXT,
               country TEXT,
               region TEXT,
               exchange TEXT)
''')


cursor.execute('''
CREATE TABLE IF NOT EXISTS daily_YF_data (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    product_index INTEGER,
    date DATE,
    open_price REAL,
    close_price REAL,
    high_price REAL,
    low_price REAL,
    volume INTEGER,
    FOREIGN KEY(product_index) REFERENCES ticker_info(product_index)
)
''')

STEP 1:
Code pour update la liste de ticker qu'il y a eu dans mon portfolio depuis sa création.

On va prendre les données de mon excel de suivi de PnL comme golden source, et on ne va pas charger les lignes ou il n'y a pas de code ISIN.
 
On va feed les isin yf_ticker et shortname depuis l'excel vers la db SQL, dans la table ticker_info, et tenter de download les autres infos via YF.
Il n'y aura pas de longName entre autres si les données n'ont pas chargé depuis yahho finance.

In [ ]:

def update_tickers(portfolio_tickers):

    yf_ticker_to_process = portfolio_tickers.dropna(subset=['YF Ticker'])
    #We don't update the excel rows where there are no YF Ticker. A modifier si le besoin change

    for _, row in yf_ticker_to_process.iterrows(): #the _ is a convention to specify we won't be using this index value _
            
            #EXCEL YFTICKER ISIN and SHORTNAME are COPY AND REPLACE vers la db SQL
            ticker = row['YF Ticker']
            short_name = row['Product Name']
            isin = row['ISIN']
            
            #ICI mettre un if l'isin est pas dedans on l'ajoute, puis on continue le code
            cursor.execute("SELECT COUNT(*) FROM ticker_info WHERE isin = ?", (isin,)) #the coma after isin to have them as a tuple
            if cursor.fetchone()[0] < 1: #True if the isin isn't in the db

                #ICI ON UPDATE LE TICKER, MAIS POUR UN NOUVEL ISIN IL FAUT AJOUTER UNE LIGNE            
                cursor.execute('''
                INSERT INTO ticker_info (YF_ticker, shortName, isin)
                VALUES (?, ?, ?)
                ''', (ticker, short_name, isin))
                conn.commit()

            
            try:
                  
                #Autres données sont re-chargées depuis YF. Si besoin d'écraser manuellement d'autres data, code à revoir ici
                long_name = yf.Ticker(ticker).info.get('longName')
                industry = yf.Ticker(ticker).info.get('industry')
                sector = yf.Ticker(ticker).info.get('sector')
                country = yf.Ticker(ticker).info.get('country')
                region = yf.Ticker(ticker).info.get('region')
                exchange = yf.Ticker(ticker).info.get('exchange')

                #ICI ON UPDATE LE TICKER, MAIS POUR UN NOUVEL ISIN IL FAUT AJOUTER UNE LIGNE            
                cursor.execute('''
                            UPDATE ticker_info
                            SET YF_ticker = ?, longName = ?, shortName = ?, industry = ?, sector = ?, country = ?, region = ?, exchange = ?
                            WHERE isin = ?
                            ''', (ticker, long_name, short_name, industry, sector, country, region, exchange, isin))
                conn.commit()

            except Exception as e:
                logging.error(f"Yahoo Finance API error for {ticker}")
                continue
    
    conn.commit() 


In [ ]:

update_tickers(portfolio_tickers)

conn.commit() 

STEP 2:
Ici on va télécharger toutes les données de yahoo finance pour les ticker de notre portfolio

On va avoir du debugging pour nous écrire les yf_ticker qui ont pas été download et le nombre de data qu'on a pour chaque ticker chargé

Par défault c'est daily closing price, de 01 01 2024 à J-2

In [ ]:
portfolio_tickers_only = pd.read_sql_query(
    "SELECT YF_ticker FROM ticker_info WHERE longName IS NOT NULL", conn
)

La table daily_close_data de la database SQL est du format :

cursor.execute('''
        CREATE TABLE IF NOT EXISTS daily_close_data (
            YF_ticker TEXT,
            Date DATE,
            Close REAL,
            PRIMARY KEY (YF_ticker, Date)
        )
    ''')

FONCTION POUR ERASE LA TABLE SQL daily_close_price

A RUN UNIQUEMENT SI ON VEUT RESET TOUTES LES DONNES YAHOO FINANCE DE NOTRE SQL TABLE daily_close_data

In [ ]:
def reset_yf_data_in_sql(database): #Uniquement si besoin de reset les data de yahoo finance enregistrées dans SQL
        cursor.execute('''
        DELETE FROM daily_close_data
        ''')

reset_yf_data_in_sql(databasename)

conn.commit()

In [ ]:
def download_yf_portfolio_tickers(portfolio_tickers_only, start_date="2024-01-01", 
                                  end_date=(datetime.now() - timedelta(days=2)).strftime('%Y-%m-%d'), 
                                  interval='1d'):
    
    # Ensure the 'daily_close_data' table exists in the database

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS daily_close_data (
            YF_ticker TEXT,
            Date DATE,
            Close REAL,
            PRIMARY KEY (YF_ticker, Date)
        )
    ''')
    conn.commit()

    # Download and update data for each ticker
    for ticker in portfolio_tickers_only['YF_ticker']:
        try:
            # Step 1: Fetch all existing dates from the yahoo finance data in our SQL, we will not upload yf data into SQL if the date is in existing_date
            cursor.execute('''
                SELECT Date FROM daily_close_data WHERE YF_ticker = ?
            ''', (ticker,))
            existing_dates = set(row[0] for row in cursor.fetchall())  # Turn into a set for unique dates

            ticker_data = yf.download(ticker, start=start_date, end=end_date, interval=interval)[['Close']]
            #La start, end date et l interval sont à présicer en appelant la fonction
            ticker_data.loc[:, 'YF_ticker'] = ticker  # Add ticker symbol to the data

            if len(ticker_data) > 0:
                ticker_data.reset_index(inplace=True)  # Reset index to make 'Date' a column
                ticker_data = ticker_data.assign(YF_ticker=ticker)  # Add ticker symbol as a column

                # Iterate over each row and insert/update only if the Date is missing
                for _, row in ticker_data.iterrows():
                    date = row.iloc[0].strftime('%Y-%m-%d') #converts timestamp to a string
                    close_price = row.iloc[1] #Using iloc rather than column name because pandas lib issues

                    if date not in existing_dates: #Si on a des dates manquantes dans le SQL pour ce ticker
                        cursor.execute('''
                            INSERT INTO daily_close_data (YF_ticker, Date, Close)
                            VALUES (?, ?, ?)
                        ''', (ticker, date, close_price))
                        print(f"Inserted data for {ticker} on {date}")
                    else: continue

        except Exception as e:
            logging.error(f"Error downloading or storing data for {ticker}: {e}")

    #DEBUGGING, reste du code de cette partie est entièrement dédiée au debugging

    #First, Checking the YF_ticker that were not loaded
    cursor.execute('''
    SELECT ticker_info.YF_ticker
    FROM ticker_info
    LEFT JOIN daily_close_data ON ticker_info.YF_ticker = daily_close_data.YF_ticker
    WHERE daily_close_data.YF_ticker IS NULL;
    ''')
    #le LEFT JOIN va prendre tous les ticker de ticker_info, et les comparer aux tickers présents dans daily_close_data (table of the LEFT), 
    #et va retourner NULL si le ticker de ticker_info n'est nul part dans daily_close_data
    # Fetch the results
    missing_tickers = cursor.fetchall()
    # Display the missing tickers in the console
    print("YF_ticker values in ticker_info but not in daily_close_data:")
    for ticker in missing_tickers:
        print(ticker[0])  # ticker is a tuple, so we access the first item

    #Second, let's print the nb of rows (date&price) we have for each downloaded ticker
    cursor.execute('''
    SELECT YF_ticker, COUNT(*) AS num_rows
    FROM daily_close_data
    GROUP BY YF_ticker;
    ''')

    print("Downloaded yf_ticker with nb of price data as below:")
    print(cursor.fetchall())

    # Commit all changes after processing the entire dataset
    conn.commit()

In [ ]:

download_yf_portfolio_tickers(portfolio_tickers_only)

conn.commit() 

STEP 3: compute log returns

Ici on va compute les log returns, en s'assurant d'avoir NULL pour la premiere date de chaque ticker

In [ ]:
def compute_log_returns(databasename):

    cursor.execute('''
        SELECT YF_ticker, Date, Close
        FROM daily_close_data
        ORDER BY YF_ticker, Date
        ''')
    data = cursor.fetchall()

    #convert to pandas for ease of use
    df_price_to_return = pd.DataFrame(data, columns=['YF_ticker', 'Date', 'Close'])
    df_price_to_return['Date'] = pd.to_datetime(df_price_to_return['Date'])

    #compute log returns
    #the df.groupby will creates groups in the data for each ('YF_ticker') and only display the ['Close'] for each group/ticker
    #the x.shift(1) 'moves' all x values to the next row/date : so we effectively substract the log of the previous day) and We get log(Pt) - log(Pt-1)
    df_price_to_return['log_return'] = (
        df_price_to_return.groupby('YF_ticker')['Close']
        .apply(lambda x: np.log(x) - np.log(x.shift(1)))
        .reset_index(level=0, drop=True)  # Reset index alignment after groupby
    )

        # Iterate over each row, skipping the first entry of each ticker (where log_return is NaN)
    for _, row in df_price_to_return.dropna(subset=['log_return']).iterrows():  # Only rows with valid log returns
        # Convert the Date to a string format
        date_str = row['Date'].strftime('%Y-%m-%d')
        cursor.execute('''
            UPDATE daily_close_data
            SET log_return = ?
            WHERE YF_ticker = ? AND Date = ?
        ''', (row['log_return'], row['YF_ticker'], date_str))

    conn.commit()
    print("Log returns computed and updated in the database.")


In [ ]:
compute_log_returns(databasename)

conn.commit()

STEP 4 (OPTIONEL)
On va vérifier le patern de nos ticker

In [ ]:
def graphiques(databasename, start_date='2024-01-01', end_date='2024-11-01'):
    start_date = '2024-01-01'
    end_date = '2024-11-01'

    # Query data for all tickers within the specified date range
    query = '''
        SELECT YF_ticker, Date, Close, log_return
        FROM daily_close_data
        WHERE Date >= ? AND Date < ?
        ORDER BY YF_ticker, Date
    '''
    df = pd.read_sql_query(query, conn, params=(start_date, end_date))
    df['Date'] = pd.to_datetime(df['Date'])

    # Get the list of unique tickers
    tickers = df['YF_ticker'].unique()

    # Plot closing price and log return for each ticker
    for ticker in tickers:
        #We have 2 df[ here because we first create a Bolean with the condition = ticker, then we apply this bolean series to return only rows of df where the bolean is true (we have YF_ticker as ticker)
        ticker_data = df[df['YF_ticker'] == ticker]

        # Create a figure with two subplots (one for Close, one for log_return)
        fig, ax = plt.subplots(2, 1, figsize=(10, 8), sharex=True)
        fig.suptitle(f'{ticker} - Closing Price and Log Return (2024)', fontsize=16)

        # Closing Price plot
        ax[0].plot(ticker_data['Date'], ticker_data['Close'], label='Closing Price')
        ax[0].set_ylabel('Closing Price')
        ax[0].set_xlim([datetime.strptime(start_date, '%Y-%m-%d'), datetime.strptime(end_date, '%Y-%m-%d')])
        ax[0].legend()

        # Log Return plot
        ax[1].plot(ticker_data['Date'], ticker_data['log_return'], color='orange', label='Log Return')
        ax[1].set_ylabel('Log Return')
        ax[1].set_xlim([datetime.strptime(start_date, '%Y-%m-%d'), datetime.strptime(end_date, '%Y-%m-%d')])
        ax[1].set_xlabel('Date')
        ax[1].legend()

        # Adjust layout and show plot
        plt.tight_layout(rect=[0, 0, 1, 0.95])
        plt.show()

        conn.commit()


In [ ]:
graphiques(databasename)

conn.commit() 

STEP 5:
On va créer l'object / class asset

Chaque instance de cet objet sera un Asset de notre portfolio. On va y calculer notre realised and unrealised pnl, et garder d autres données comme close_price ou les transactions cost dedans.

On va créer un autre pandas dataframe ou on va append toutes ces données à chaque instance de Asset, pour avoir toutes les données de notre portfolio dans ce df.

A la fin de chaque instance de Asset, on va calculer le pnl total de notre portfolio (à améliorer, pas besoin de faire ce calcul a la fin de chaque instance mais uniquement à la dernière instance)

Asset Class:

We have fixed values in our Asset obkect : shortname, ticker

Other values varies with time, we create a dictionnary to store all these data for each date.

We will compute quantity, position size, realized and unrealized pnl, and get close prices and transaction costs in the dictionary

Note that we need to refresh our YF database before running the pnl

We first select the ticker or name of the asset, and it will get all the trading_dates in our SQL database. For each date, we initialize the dictionnary. If there is a transaction or more on this date in our excel, we iterate through it and modify our variables accordingly (quantity etc).
If there are no transactions, we  just update the relevant metric (unrealized_pnl, position) 
We create plots to visualize these metrics


It would be best to create another object named Portfolio, or to compute the total portfolio metrics outside of the loop in the Asset object, need to improve that

It would also be best not to plot all the charts in the loop, but to be able to print the ones we are interested in

In [ ]:
class Asset:
    def __init__(self, ticker, shortname):
        # Fixed attributes
        self.ticker = ticker
        self.shortname = shortname 
        
        # Filter the provided transactions DataFrame to only include transactions for this asset (by ISIN)
        self.transactions = transactions[transactions['Product'] == shortname].copy()
        #'Product' contains shortname values in our spreadsheet

        # Variables that change over time
        self.quantity = 0
        self.close_price = 0
        self.position_eur = 0
        self.inventory_wac_cost = 0
        self.total_invested = 0
        self.realized_pnl = 0
        self.unrealized_pnl = 0
        self.transaction_cost = 0
        self.total_pnl = 0

        # Dictionary to store time-dependent data by date
        self.daily_data = {}
        #Initialize the pd df we will use to compute portfolio metrics
        portfolio_metrics = pd.DataFrame()
        portfolio_data = pd.DataFrame()


    def process_ticker_data(self):
        #Penser a changer la SQL table si on change la granularité de temps
        query = '''
        SELECT DISTINCT Date
        FROM daily_close_data
        ORDER BY Date
        '''
        cursor.execute(query)

        # Fetch the results and create a sorted set of unique dates
        trading_dates = sorted(pd.to_datetime(row[0]) for row in cursor.fetchall())

        for date in trading_dates:

            # Initialize daily_data with default values
            self.daily_data[date] = {
                    'shortname' : self.shortname,
                    'quantity': self.quantity,
                    'position_eur': self.position_eur,
                    'close_price' : self.close_price,
                    'inventory_wac_cost': self.inventory_wac_cost,
                    'total_invested' : self.total_invested,
                    'transaction_cost': self.transaction_cost,
                    'realized_pnl': self.realized_pnl,
                    'unrealized_pnl': self.unrealized_pnl,
                    'total_pnl' : self.total_pnl
            }

            transactions_this_day = self.transactions[
            (self.transactions['Date'] == date) & (self.transactions['Product'] == self.shortname)
            ]

            # If there are transactions on this date, iterate through them and compute metrics
            if not transactions_this_day.empty:
                for _, transaction in transactions_this_day.iterrows():
                    quantity = transaction['Quantity']
                    share_price = transaction['Share price']
                    transaction_type = transaction['Transaction type']
                    position = transaction['Amount (EUR equiv)']
                    transaction_cost = transaction['Transaction Cost']

                    if transaction_type == 'Buy':
                        self.total_invested += position
                        self.position_eur += position
                        self.quantity += quantity
                        self.inventory_wac_cost = self.total_invested / self.quantity if self.quantity > 0 else 0
                        self.unrealized_pnl = self.position_eur - self.total_invested 

                    elif transaction_type == 'Sell':
                        self.realized_pnl += position - (self.inventory_wac_cost * quantity) #et pas self.quantity sinon on prend tous le stock et pas juste la transaction
                        self.quantity -= quantity
                        self.position_eur -= position 
                        self.total_invested -= (self.inventory_wac_cost * quantity)
                        self.unrealized_pnl = self.position_eur - self.total_invested 
                        if self.quantity ==0:
                            self.inventory_wac_cost = 0


                    elif transaction_type == 'Div and interests':
                        self.realized_pnl += position
                        continue

                    elif transaction_type == 'Cash transfer':
                        continue

                    self.transaction_cost += transaction_cost
                    print(f"A transaction for {self.shortname} has been processed on the {date}")


            #GET THE CLOSE PRICE FROM SQL YF DATABASE
            query_price = '''
            SELECT Close
            FROM daily_close_data
            WHERE YF_ticker = ? AND Date <= ?
            ORDER BY Date DESC
            LIMIT 1
            '''
            #LIMIT 1 is to get the closest value if there are none close price for that date
            cursor.execute(query_price, (self.ticker, date.strftime('%Y-%m-%d')))
            #SQLite doesn"t support timestamp format
            result = cursor.fetchone()
            close_price = result[0] if result else 0

            if close_price != 0: #seulement si on a un close price
                self.position_eur = self.quantity * close_price
                self.unrealized_pnl = (close_price - self.inventory_wac_cost) * self.quantity
            else:
                self.unrealized_pnl = 0 #On ne calcule pas de Unrealized PNL si on a pas de daily data de YF

            self.total_pnl = self.unrealized_pnl + self.realized_pnl

            #we store the daily data in this dictionnary at each EOD. We will then use these data to compute the portfolio metrics
            self.daily_data[date].update({
                    'shortname' : self.shortname,
                    'quantity': self.quantity,
                    'position_eur': self.position_eur,
                    'close_price' : close_price,
                    'inventory_wac_cost': self.inventory_wac_cost,
                    'total_invested' : self.total_invested,
                    'transaction_cost': self.transaction_cost,
                    'realized_pnl': self.realized_pnl,
                    'unrealized_pnl': self.unrealized_pnl,
                    'total_pnl' : self.total_pnl
            })

            #print(f"(The {self.shortname} data for the {date} are {close_price} close price, {self.inventory_wac_cost} in inventory WAC, {self.unrealized_pnl} unrealized pnl, {self.realized_pnl} realized pnl, and {self.quantity} number of shares")

        #Once all dates have been processed, we delete the first row of our dictionary as it is empty
        first_key = list(self.daily_data.keys())[0]  # Get the first key in the dictionary
        del self.daily_data[first_key]

        df_last_ticker = pd.DataFrame.from_dict(self.daily_data, orient='index')
        df_last_ticker.index.name = 'Date'
        df_last_ticker.reset_index(inplace=True) #moves the dates from being an index to a regular column

        #We plot a chart just to check consistency
        plt.figure(figsize=(12, 6))
        plt.plot(df_last_ticker['Date'], df_last_ticker['realized_pnl'], label='Realized PnL', marker='o')
        plt.plot(df_last_ticker['Date'], df_last_ticker['unrealized_pnl'], label='Unrealized PnL', marker='x')
        plt.plot(df_last_ticker['Date'], df_last_ticker['total_pnl'], label='Total PnL', marker='s')
        plt.xlabel('Date')
        plt.ylabel('PnL')
        plt.title(f'Realized, Unrealized and Total PnL for {self.shortname}')
        plt.legend()
        plt.grid()
        plt.show()

        global portfolio_data, portfolio_metrics #Need to reset this dataframe before calling the functions as it is a global variable that will keep the data stored
        portfolio_data = pd.concat([portfolio_data, df_last_ticker], ignore_index=True)
        print(f"We have added {self.shortname} pnl to our portfolio data df")

        portfolio_metrics['shortname'] = "TOTAL_PORTFOLIO"
        portfolio_metrics = (
        portfolio_data.groupby('Date')[['total_pnl', 'realized_pnl', 'unrealized_pnl', 'position_eur', 'transaction_cost']]
        .sum()
        .reset_index()
        )



In [ ]:

portfolio_data = pd.DataFrame() #resets the portfolio data as it is a global variable
portfolio_metrics = pd.DataFrame() 
individual_asset = {}

for _, row in portfolio_tickers.iterrows():
    shortname = row.iloc[0]
    yf_ticker = row.iloc[3] 
    individual_asset[shortname] = Asset( ticker=yf_ticker,shortname = shortname)
    individual_asset[shortname].process_ticker_data()

conn.commit()

In [ ]:
# Set up the figure
plt.figure(figsize=(12, 6))

    # Plot realized PnL
plt.plot(portfolio_metrics['Date'], portfolio_metrics['realized_pnl'], label='Realized PnL', marker='o')
plt.plot(portfolio_metrics['Date'], portfolio_metrics['unrealized_pnl'], label='Unrealized PnL', marker='x')
plt.plot(portfolio_metrics['Date'], portfolio_metrics['total_pnl'], label='Total PnL', marker='s')

    # Add labels and title
plt.xlabel('Date')
plt.ylabel('PnL')
plt.title('Portfolio Realized, Unrealized and Total PnL Over Time')
plt.legend()
plt.grid()

    # Display the plot
plt.show()

In [ ]:
conn.close()